In [49]:
import os
import re
import string
import email

In [60]:
def email_to_name(text):
    import email
    import os
    import re

    if "@" not in text:
        return text
    
    name, mail = email.utils.parseaddr(text)
    if name:
        return name
    
    name, domain = text.split("@")
    name_parts = [x.strip() for x in re.split(r"@|\.|\W|\_", name) if x.strip()]
    return " ".join(name_parts)

In [58]:
unique_senders = set()
for root, dirs, files in os.walk(r"F:\Documents\stopansko\masters\thesis\sig-detect\data\clean\enron_random_clean"):
    for file in files:
        if "sender" in file:
            with open(os.path.join(root, file), encoding="utf-8") as f:
                mail = f.read()           
                name = email_to_name(mail)
                unique_senders.add(name)

In [90]:
def has_sender_name(line, sender_name):
    for name in sender_name.split(" "):
        if name in line:
            return True
    return False

# Add Senders

In [63]:
import pandas as pd

In [64]:
master = pd.read_pickle("enron_random_clean1.pkl")

In [67]:
filenames = master.filename.unique()

In [71]:
senders = {}
for filename in filenames:
    base_path = r"F:\Documents\stopansko\masters\thesis\sig-detect\data\clean\enron_random_clean"
    with open(os.path.join(base_path, filename.replace(".txt", "_sender.txt")), encoding="utf-8") as f:
        senders[filename] = f.read()

In [74]:
master["sender"] = master.apply(lambda row: senders[row.filename], axis=1)
master["sender_name"] = master.apply(lambda row: email_to_name(senders[row.filename]), axis=1)
master["has_sender"] = master.apply(lambda row: row.sender.lower() in row.line.lower(), axis=1)
master["has_sender_name"] = master.apply(lambda row: 1 if has_sender_name(row.line, row.sender_name) else 0, axis=1)

In [97]:
master.to_pickle("enron_random_clean1_senders.pkl")